In [19]:

import json
import numpy as np
import pandas as pd
from getpass import getpass, getuser

import requests
from requests.auth import HTTPBasicAuth

# credentials = json.loads(open('Sec\credentials.json').read())
user=input("Enter your github Username")
passw=getpass()
authentication = HTTPBasicAuth(user, passw)
#We authorize our requests using this authentication variable (equivalen to auth=(username,password) in a request)

data = requests.get('https://api.github.com/users/' + user,
                    auth = authentication)
data = data.json()
pretty_data=json.dumps(data,indent=4)
# print(pretty_data)
user_name=data['name']
user_location=data['location']
user_email=data['email']
user_bio=data['bio']
user_public_repos_num=data['public_repos']
user_repos_url=data['repos_url'] #provides the github API to fetch the repository details for the user

repos_data=[]# This is where we will store all our 
tot_repos_fetched=0
#Since the given api only fetches about 30 repositories we use a seperatre parameter known as page numbers to fetch the further repos
while (True):
    response = requests.get(user_repos_url, auth = authentication)
    response = response.json()
    repos_data = repos_data + response
    repos_fetched = len(response)
    tot_repos_fetched+=repos_fetched
    print("Total repositories fetched: {}".format(repos_fetched))
    if (repos_fetched == 30):
        page_no = page_no + 1
        url = data['repos_url'].encode("UTF-8") + '?page=' + str(page_no)
    else:
        break

current_repo=repos_data[0]
pretty_repo=json.dumps(current_repo,indent=4)
repo_name=current_repo['name']
repo_description=current_repo['description']

#? We create another request to get the languages used ( we get a json file with number of lines present)
repo_language_url=current_repo['languages_url']

language=requests.get(repo_language_url)
print(type(language))
language=language.json()
#? To get an idea of what the repo looks like
# print(pretty_repo)



Total repositories fetched: 10
<class 'requests.models.Response'>


In [22]:
from asyncio.windows_events import NULL
import csv

filename="repos_all.csv"
fields=["repo_name","repo_description"]
# if str(repo_description)=='none':
#     repo_description=""
print(repo_name)
rows=[[repos_data[i]["name"],repos_data[i]["description"]] for i in range(tot_repos_fetched)]
print(rows)

with open(filename,'w')as csvfile:
    csvwriter = csv.writer(csvfile)
   
    # writing the fields
    csvwriter.writerow(fields)
     
    # writing the data rows
    csvwriter.writerows(rows)

CalHousePredictor
[['CalHousePredictor', None], ['Class_Prediction_with-given_Features_ML', None], ['Codewars', None], ['Java-learning', 'syntax and statement definitions with examples in Java'], ['learning-C-with-classes', 'Reference C++ scripts.'], ['Mayhul-Jindal', 'My GitHub Profile'], ['NLP_pub', None], ['Profit_MLmodel', None], ['python-scripts', None], ['Whatsapp-Bot', 'A digital marketing tool that can be used to send product updates, message people individually without the creation of extra groups using Whatsapp']]
